# Arithmetik

Eine wichtige Funktion von pandas ist das arithmetische Verhalten bei Objekten mit unterschiedlichen Indizes. Wenn beim Addieren von Objekten die Indexpaare nicht gleich sind, wird der entsprechende Index im Ergebnis die Vereinigung der Indexpaare sein. Für Benutzer mit Datenbankerfahrung ist dies vergleichbar mit einem automatischen [OUTER JOIN](https://de.wikipedia.org/wiki/Join_(SQL)#%C3%84u%C3%9Ferer_Verbund_(OUTER_JOIN)) auf den Indexbezeichnungen. Schauen wir uns ein Beispiel an:

In [1]:
import numpy as np
import pandas as pd

s1 = pd.Series(np.random.randn(5))
s2 = pd.Series(np.random.randn(7))

Addiert man diese Werte, erhält man:

In [2]:
s1 + s2

0   -0.117652
1    1.843403
2   -1.554283
3   -0.052506
4   -0.248629
5         NaN
6         NaN
dtype: float64

Der interne Datenabgleich führt zu fehlenden Werten an den Stellen der Labels, die sich nicht überschneiden. Fehlende Werte werden dann bei weiteren arithmetischen Berechnungen weitergegeben.

Bei DataFrames wird die Ausrichtung sowohl für die Zeilen als auch für die Spalten durchgeführt:

In [3]:
df1 = pd.DataFrame(np.random.randn(5,3))
df2 = pd.DataFrame(np.random.randn(7,2))

Wenn die beiden DataFrames addiert werden, ergibt sich ein DataFrame, dessen Index und Spalten die Vereinigungen derjenigen in jedem der obigen DataFrames sind:

In [4]:
df1 + df2

,0,1,2
0,-1.617414,1.480734,NaN
1,1.060217,2.245327,NaN
2,0.604860,-1.533916,NaN
3,-0.487990,-1.244289,NaN
4,-2.641149,-1.044312,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN


Da die Spalte 2 nicht in beiden DataFrame-Objekten vorkommen, erscheinen sie im Ergebnis als fehlend. Das Gleiche gilt für die Zeilen, deren Bezeichnungen nicht in beiden Objekten vorkommen.

## Arithmetische Methoden mit Füllwerten

Bei arithmetischen Operationen zwischen unterschiedlich indizierten Objekten kann es sinnvoll sein, einen speziellen Wert (z. B. `0`) zu verwenden, wenn eine Achsenbeschriftung in einem Objekt gefunden wird, im anderen aber nicht.  Mit der `add`-Methode kann das Argument `fill_value` übergeben werden:

In [5]:
df12 = df1.add(df2, fill_value=0)

df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,NaN
6,1.864685,1.405582,NaN


Im folgenden Beispiel setzen wir die beiden verbleibenden NaN-Werte auf `0`:

In [6]:
df12.iloc[[5,6], [2]] = 0

In [7]:
df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,0.000000
6,1.864685,1.405582,0.000000


## Arithmetische Methoden

Methode | Beschreibung
:------ | :-----------
`add`, `radd` | Methoden für Addition (`+`)
`sub`, `rsub` | Methoden für die Subtraktion (`-`)
`div`, `rdiv` | Methoden für die Division (`/`)
`floordiv`, `rfloordiv` | Methoden für die Abrundungsfunktion (engl.: floor divison) (`//`)
`mul`, `rmul` | Methoden für die Multiplikation (`*`)
`pow`, `rpow` | Methoden zur Potenzierung (`**`)

`r` (engl.: _reverse_) kehrt die Methode um.

## Operationen zwischen DataFrame und Series

Wie bei NumPy-Arrays verschiedener Dimensionen ist auch die Arithmetik zwischen DataFrame und Series definiert.

In [8]:
s1 + df12

,0,1,2,3,4
0,-3.413133,2.531767,-0.858270,NaN,NaN
1,-0.735503,3.296360,-1.001270,NaN,NaN
2,-1.190859,-0.482883,-1.138355,NaN,NaN
3,-2.283709,-0.193256,-1.718093,NaN,NaN
4,-4.436868,0.006721,0.484069,NaN,NaN
5,-2.451955,-0.862252,-0.513864,NaN,NaN
6,0.068966,2.456614,-0.513864,NaN,NaN


Wenn wir `s1` mit `df12` addieren, wird die Additon für jede Zeile einmal durchgeführt. Dies wird als _Broadcasting_ bezeichnet. Standardmäßig entspricht die Arithmetik zwischen DataFrame und Serie dem Index der Serie in den Spalten des DataFrame, wobei die Zeilen nach unten übertragen werden.

Wenn ein Indexwert weder in den Spalten des DataFrame noch im Index der Serie gefunden wird, werden die Objekte neu indiziert, um die Vereinigung zu bilden:

Wenn ihr stattdessen die Spalten übertragen und die Zeilen abgleichen wollt, müsst ihr eine der arithmetischen Methoden verwenden, z.B.:

In [9]:
df12.add(s2, axis='index')

,0,1,2
0,0.060653,3.158801,1.333661
1,1.852586,3.037697,0.304963
2,-0.435558,-2.574335,-1.664910
3,0.723141,-0.033158,0.006902
4,-2.536814,-0.939976,1.102269
5,-1.370872,-2.627920,-0.714636
6,0.512718,0.053615,-1.351967


Die Achsennummer, die ihr übergebt, ist die Achse, auf die abgeglichen werden soll. In diesem Fall soll der Zeilenindex des DataFrame (`axis='index'` oder `axis=0`) abgeglichen und übertragen werden.

## Funktionsanwendung und Mapping

`numpy.ufunc` (elementweise Array-Methoden) funktionieren auch mit Pandas-Objekten:

In [10]:
np.abs(df12)

,0,1,2
0,1.617414,1.480734,0.344406
1,1.060217,2.245327,0.487406
2,0.604860,1.533916,0.624491
3,0.487990,1.244289,1.204230
4,2.641149,1.044312,0.997933
5,0.656236,1.913285,0.000000
6,1.864685,1.405582,0.000000


Eine weitere häufige Operation ist die Anwendung einer Funktion auf eindimensionale Arrays auf jede Spalte oder Zeile. Die [pandas.DataFrame.apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)-Methode tut genau dies:

In [11]:
df12

,0,1,2
0,-1.617414,1.480734,-0.344406
1,1.060217,2.245327,-0.487406
2,0.604860,-1.533916,-0.624491
3,-0.487990,-1.244289,-1.204230
4,-2.641149,-1.044312,0.997933
5,-0.656236,-1.913285,0.000000
6,1.864685,1.405582,0.000000


In [12]:
f = lambda x: x.max() - x.min()

df12.apply(f)

0    4.505834
1    4.158612
2    2.202163
dtype: float64

Hier wird die Funktion `f`, die die Differenz zwischen dem Maximum und dem Minimum einer Reihe berechnet, einmal für jede Spalte des Rahmens aufgerufen. Das Ergebnis ist eine Reihe mit den Spalten des Rahmens als Index.

Wenn ihr `axis='columns'` an `apply` übergebt, wird die Funktion stattdessen einmal pro Zeile aufgerufen:

In [13]:
df12.apply(f, axis='columns')

0    3.098148
1    2.732734
2    2.138776
3    0.756299
4    3.639082
5    1.913285
6    1.864685
dtype: float64

Viele der gebräuchlichsten Array-Statistiken (wie `sum` und `mean`) sind DataFrame-Methoden, so dass die Verwendung von `apply` nicht notwendig ist.

Die an apply übergebene Funktion muss keinen Einzelwert zurückgeben; sie kann auch eine Reihe mit mehreren Werten zurückgeben:

In [14]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

df12.apply(f)

,0,1,2
min,-2.641149,-1.913285,-1.204230
max,1.864685,2.245327,0.997933


Es können auch elementweise Python-Funktionen verwendet werden. Angenommen, ihr möchtet aus jedem Fließkommawert in `df12` eine formatierte Zeichenkette berechnen. Dies könnt ihr mit [pandas.DataFrame.applymap](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html) erreichen:

In [15]:
f = lambda x: '%.2f' % x

df12.applymap(f)

,0,1,2
0,-1.62,1.48,-0.34
1,1.06,2.25,-0.49
2,0.60,-1.53,-0.62
3,-0.49,-1.24,-1.20
4,-2.64,-1.04,1.00
5,-0.66,-1.91,0.00
6,1.86,1.41,0.00


Der Grund für den Namen `applymap` ist, dass Series über eine `map`-Methode zur Anwendung einer elementweisen Funktion verfügt:

In [16]:
df12[2].map(f)

0    -0.34
1    -0.49
2    -0.62
3    -1.20
4     1.00
5     0.00
6     0.00
Name: 2, dtype: object